# Deploy MIRA Small

A lot of the steps below are taken from [this blog post](https://aws.amazon.com/blogs/machine-learning/deploy-trained-keras-or-tensorflow-models-using-amazon-sagemaker/) which explains how to take advantage of Amazon SageMaker deployment capabilities, such as selecting the type and number of instances, performing A/B testing, and Auto Scaling. Auto Scaling clusters are spread across multiple Availability Zones to deliver high performance and high availability.

In this notebook we'll be deploying the Small MIRA model, or "Ratter", which was trained to detect smaller Santa Cruz Island animals (classes are "Rodent", "Empty"). The Keras model graph and weight files can be downloaded [here](https://github.com/tnc-ca-geo/mira/tree/master/mira_models).

### Step 1. Set up

If you're already reading this in a Sagemaker Notebook instance, just execute the code block below to get the Sagemaker execution role.

If not, and you need to set up the Sagemaker Notebook, in the AWS Management Console go to the Amazon SageMaker console. We are using Elastic Inference, which is not available in all regions, so make sure you're in a region that supports it (e.g. us-west-2). From the console, choose Notebook Instances, and create a new notebook instance. Associate it with the animl-ml git repo (https://github.com/tnc-ca-geo/animl-ml), and set the kernel to conda_tensorflow_p36.

The ```get_execution_role``` function retrieves the AWS Identity and Access Management (IAM) role you created at the time of creating your notebook instance.

In [ ]:
import boto3, re
from sagemaker import get_execution_role

role = get_execution_role()

### Step 2. Load the Keras model using the JSON and weights file
If you've alredy converted the Keras model and prepared the ```export/``` directory, you can skip this and jump to step 5.

In [ ]:
import keras
from keras.models import model_from_json

!mkdir keras_model

Download the json file ([model-ratter-20181130.json](https://github.com/tnc-ca-geo/mira/blob/master/mira_models/model-ratter-20181130.json)) and model weights ([model-ratter-20181130.hdf5](https://github.com/tnc-ca-geo/mira/blob/master/mira_models/model-ratter-20181130.hdf5)) from the MIRA repo mentioned above. Navigate to ```keras_model``` from the Jupyter notebook home, and upload both model.json and model-weights.h5 files (using the “Upload” menu on the Jupyter notebook home).

In [ ]:
!ls keras_model

json_file = open('/home/ec2-user/SageMaker/animl-ml/notebooks/keras_model/'+'model-ratter-20181130.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights('/home/ec2-user/SageMaker/animl-ml/notebooks/keras_model/model-ratter-20181130.hdf5')

print("Loaded model from disk")

### Step 3. Export the Keras model to the TensorFlow ProtoBuf format

In [ ]:
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants

# Note: This directory structure will need to be followed - see notes for the next section
model_version = '1'
export_dir = 'export/Servo/' + model_version

# Build the Protocol Buffer SavedModel at 'export_dir'
builder = builder.SavedModelBuilder(export_dir)

# Create prediction signature to be used by TensorFlow Serving Predict API
print('loaded_model.input: {}'.format(loaded_model.input))
print('loaded_model.output: {}'.format(loaded_model.output))
signature = predict_signature_def(
    inputs={"inputs": loaded_model.input}, outputs={"score": loaded_model.output})

from keras import backend as K

with K.get_session() as sess:
    # Save the meta graph and variables
    builder.add_meta_graph_and_variables(
        sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
    builder.save()

### Step 4. Copy pre & post processing scrips into export directory

Copy inference.py and requirements.txt into ```export/code/```. 

```inference.py``` is a pre/post processing script, and its dependencies in ```requirements.txt``` get installed in the endpoint container when it is initialized. More on that and examples [here](https://github.com/aws/sagemaker-tensorflow-serving-container#prepost-processing).

In [ ]:
!cp -r ../models/mira-small/code export/

Your export directory structure should now look like this:

```
notebooks
     ...
     ├─ deploy-mira-small.ipynb
     ├─ export
           └─ Servo
                 └─ 1
                       └─ saved_model.pb
                       └─ variables
                           └─ variables.data-00000-of-00001
                           └─ variables.index
           └─ code
                 └─ inference.py
                 └─ requirements.txt
```

### Step 5. Tar the entire directory and upload to S3
Yeeehaw now we're read to zip it all up and upload it to s3...

In [ ]:
import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')

Do A quick sanity check to make sure the boto region is on in which Elastic Inference is supported (e.g. us-west-2)

In [ ]:
print(sagemaker_session.boto_region_name)

### Step 6. Deploy the trained model

There are [two ways to deploy models to sagemaker](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-deploy-model.html), using the AWS Sagemaker Python SDK (what we use below), or using the AWS SDK for Python (Boto 3). Boto 3 offers more lower level configuration controls. Documentation on using the Sagemaker Python SDK for deployment can be found [here](https://sagemaker.readthedocs.io/en/stable/using_tf.html#deploy-to-a-sagemaker-endpoint). The ```model.deploy()``` function returns a predictor that you can use to test inference on right away.

In [ ]:
endpoint_name = 'mira-small'

In [ ]:
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                        name = endpoint_name,
                        role = role,
                        framework_version = '1.14',
                        entry_point = 'inference.py',
                        source_dir='export/code'
                       )

In [ ]:
%%time
predictor = sagemaker_model.deploy(initial_instance_count = 1,
                                   instance_type = 'ml.c5.xlarge',
                                   accelerator_type='ml.eia2.medium',
                                   endpoint_name = endpoint_name
                                   )

### Step 7. Invoke the endpoint

In [ ]:
import os
from matplotlib.pyplot import imshow
from io import BytesIO
import json
import boto3
import numpy as np
from PIL import Image

HOME = os.path.expanduser('~')
CLASSES = ['rodent', 'empty']
TEST_INPUT = [
    {
        'file': 'SageMaker/animl-ml/input/sample-img.jpg',
        'detections': [{
            'category': '1',
            'conf': 0.970706,
            'bbox': [0.536007, 0.434649, 0.635773, 0.543599]
        }]
    },
    {
        'file': 'SageMaker/animl-ml/input/sample-img-fox.jpg',
        'detections': [{
            'category': '1',
            'conf': 0.970706,
            'bbox': [0.24598, 0.475871, 0.567701, 0.636399]
        }]
    },
    {
        'file': 'SageMaker/animl-ml/input/sample-img-fox-2.jpg',
        'detections': [{
            'category': '1',
            'conf': 0.970706,
            'bbox': [0.229707, 0.256563, 0.646652, 0.542365]
        }]
    },
    {
        'file': 'SageMaker/animl-ml/input/sample-img-rodent.jpg',
        'detections': [{
            'category': '1',
            'conf': 0.99954,
            'bbox': [0.0580928, 0.383797, 0.409989, 0.663712]
        }]
    }
]

client = boto3.client('runtime.sagemaker')

# prep image
image = Image.open(os.path.join(HOME, TEST_INPUT[3]['file']))
# imshow(np.asarray(image))

# Megadetector bbox is [ymin, xmin, ymax, xmax] in relative values
# convert to tuple (xmin, ymin, xmax, ymax) in pixel values 
W, H = image.size
box = TEST_INPUT[3]['detections'][0]['bbox']
boxpx = (int(box[1]*W), int(box[0]*H), int(box[3]*W), int(box[2]*H))
crp = image.crop(boxpx)
imshow(np.asarray(crp))

# convert to bytes
buf = BytesIO()
crp.save(buf, format='JPEG')

# invoke endpoint
response = client.invoke_endpoint(
    EndpointName = endpoint_name,
    ContentType = 'application/x-image', 
    Body = buf.getvalue()
)

# parse response
response_body = response['Body'].read()
response_body = response_body.decode('utf-8')
pred = json.loads(response_body)['predictions'][0]

output = {}
for i in range(len(pred)): output[CLASSES[i]] = float(pred[i])
    
print(output)